In [137]:
import pandas as pd

IRRELEVANT_COLUMNS = ['rentfaster_id', 'address', 'link', 'availability_date', 'province', 'city']
LIVING_SPACE_TYPES = ['Apartment', 'Condo Unit', 'Basement', 'House', 'Townhouse', 'Main Floor', 'Room For Rent', 'Duplex', 'Loft', 'Mobile', 'Vacation Home']
RENT_PRICE_THRESHOLD = 2000

df = pd.read_csv('datasets/dataset_joined.csv')

df = df.drop_duplicates()
df = df.drop(columns=IRRELEVANT_COLUMNS)
df = df[df['type'].isin(LIVING_SPACE_TYPES)] # filter out non-living spaces

# feature extraction
df['beds'] = df['beds'].replace('Studio', '0')
df['beds'] = df['beds'].str.extract(r'(\d+)') # extract bed count from string

df['baths'] = df['baths'].replace('none', 0)

df['sq_feet'] = df['sq_feet'].replace(',', '')
df['sq_feet'] = df['sq_feet'].str.extract(r'(\d+)') # extract square footage from string

# remove rows with missing data, since we have a lot of data points
df = df.dropna()

df['beds'] = df['beds'].astype(int)
df['baths'] = df['baths'].astype(float)
df['cats'] = df['cats'].astype(int)
df['dogs'] = df['dogs'].astype(int)
df['sq_feet'] = df['sq_feet'].astype(int)
df['price'] = df['price'].astype(float)

# one-hot encoding on categorical columns
categorical_columns = ['lease_term', 'furnishing', 'type', 'smoking']
for column in categorical_columns:
    one_hot = pd.get_dummies(df[column], prefix=column, dtype=int)
    df = df.join(one_hot)
df = df.drop(columns=categorical_columns)

# z-score standardization on numerical values
numerical_columns = ['latitude', 'longitude', 'beds', 'baths', 'sq_feet', 'nearestSchool', 'nearestPolice', 'restaurantCount', 'cemeteryCount', 'healthCareCount', 'communityCentreCount', 'placeOfWorshipCount', 'leisureCount', 'shopCount', 'tourismCount']
for column in numerical_columns:
    if df[column].std() == 0:
        continue
    df['{}_standardized'.format(column)] = (df[column] - df[column].mean()) / df[column].std()

# convert target variable into class
df['price'] = df['price'] >= RENT_PRICE_THRESHOLD
df['price'] = df['price'].replace({ True: ">=2000", False: "<2000" })

df.to_csv('datasets/dataset_cleaned.csv', index=False)